In [ ]:
# ======================INICIO=DO=STATUS===========================================
# 1. Instala as bibliotecas necessárias
!pip install requests beautifulsoup4 --quiet


import requests
from bs4 import BeautifulSoup
import re

def get_situacao_processo_web(processo_id: str) -> str:
    """
    Busca a "Situação" de um processo no portal PROA (web scraping).

    Args:
        processo_id: O número do processo (ex: "24190000141650").

    Returns:
        A situação (ex: "Ativo") se encontrado.
        Uma string de erro (ex: "ERRO: Não encontrado") se falhar.
    """

    # 1. Configuração da Requisição
    base_url = "https://secweb.procergs.com.br/pra-aj4/public/proa_retorno_consulta_publica.xhtml"
    params = {"numeroProcesso": processo_id}
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }

    # Valor padrão em caso de falha
    situacao_padrao = "ERRO: Não encontrado"

    try:
        # 2. Faz a requisição HTTP
        response = requests.get(base_url, params=params, headers=headers, timeout=10)

        # Levanta um erro se a página não for 200 (OK)
        response.raise_for_status()

        # 3. Analisa o HTML
        soup = BeautifulSoup(response.text, 'html.parser')

        # 4. Lógica de extração
        # Encontra a tag <label> que contém o texto "Situação:"
        situacao_label_tag = soup.find('label', string=re.compile(r"Situação:"))

        if situacao_label_tag:
            # Tenta navegar na tabela para achar o valor
            try:
                parent_td = situacao_label_tag.find_parent('td')
                value_td = parent_td.find_next_sibling('td')
                situacao_valor = value_td.get_text(strip=True)

                # Retorna o valor limpo se não for vazio
                return situacao_valor if situacao_valor else situacao_padrao

            except Exception:
                # Falha ao navegar na estrutura (ex: HTML mudou)
                return "ERRO: Falha no parse do HTML"
        else:
            # Não encontrou o label "Situação:" na página
            return situacao_padrao

    except requests.exceptions.HTTPError:
        # Erro HTTP (ex: 404 - Não Encontrado, 500 - Erro de Servidor)
        return "ERRO: Página não encontrada ou servidor falhou"
    except requests.exceptions.RequestException:
        # Erro de rede (ex: DNS, Timeout, sem internet)
        return "ERRO: Falha na conexão"

# ======================FIM=DO=STATUS===========================================

!pip install gspread gspread-dataframe google-auth --quiet
!pip install pymupdf --quiet

# 2. Rode esta célula para autenticar sua conta no Colab
# Isso permitirá que o script acesse seus arquivos do Google
from google.colab import auth

auth.authenticate_user()

# 3. Configure o cliente do gspread
import gspread
from google.auth import default

creds, _ = default()
gc = gspread.authorize(creds)

# ==========================
# CONFIGURAÇÃO INICIAL
# ==========================
import os
import re
import datetime
import pandas as pd
import fitz  # pymupdf
import tiktoken

# ====== IMPORTAÇÕES (Google Sheets) ======
import gspread
from gspread_dataframe import get_as_dataframe, set_with_dataframe
from google.colab import auth
from google.auth import default

# ====== CAMINHOS IMPORTANTES ======
PDF_DIR = "/content/drive/MyDrive/processos_cibelle"  # pasta com os PDFs

# ====== CONFIGURAÇÃO DO GOOGLE SHEETS ======
# O NOME da sua planilha no Google Drive
GSHEET_NAME = "Cibelle_automation"
# O NOME da aba (worksheet) dentro da planilha
GSHEET_WORKSHEET_NAME = "TABELA"

# ======= MENSAGENS DE ERROS ========

ERR_MSG_EXPEIDENTE = "Sem Penalidade"
ERR_MSG_TIPO_PENALIDADE  = ""
ERR_MSG_PERCENTUAL_MULTA  = ""
ERR_MSG_IMPEDIMENTOS = ""
ERR_MSG_PENALIDADE_MESES = ""
ERR_MSG_DATA_PENALIZACAO = ""
ERR_MSG_STATUS = "ERRO: IMPOSSIVEL DE DEFINIR UM STATUS"
# ====== NOME DAS COLUNAS PADRÃO ======
COLUMNS = [
    "numero_contrato",
    "nome_empresa",
    "cnpj_empresa",
    "proa_notificatorio",
    "proa_mae",
    "status_processo",
    "valor_contrato_consolidado",
    "tipo_penalidade",
    "percentual_multa",
    "valor_multa",
    "impedimentos",
    "penalidade_meses",
    "data_penalizacao",
    "ultima_analise_feita",
    "ultima_atualizacao_processo",
    #"token_input_consumo",
]

# ==========================
# FUNÇÃO: LER TEXTO DO PDF
# ==========================
def extract_pdf_text(pdf_path: str) -> str:
    """Extrai TODO o texto do PDF página a página e concatena em uma string única."""
    doc = fitz.open(pdf_path)
    pages_text = []
    for page in doc:
        pages_text.append(page.get_text("text"))
    full_text = "\n".join(pages_text)
    doc.close()
    return full_text


# ==========================
# FUNÇÃO: ESTIMAR TOKEN
# ==========================
def estimate_tokens(text: str, model_name: str = "gpt-4o-mini"):
    """
    Estima tokens de entrada de acordo com a tokenização tiktoken.
    Use um modelo aproximado (ex: gpt-4o-mini ou gpt-4o).
    Ajuste pro modelo real que você usar na API.
    """
    try:
        enc = tiktoken.encoding_for_model(model_name)
    except Exception:
        # fallback genérico
        enc = tiktoken.get_encoding("cl100k_base")
    return len(enc.encode(text))


# ==================================
# FUNÇÕES DE EXTRAÇÃO (REGEX)
# (Estas funções permanecem IDÊNTICAS)
# ==================================

def _clean_company_name(s: str) -> str:
    # colapsa quebras de linha / espaços múltiplos e remove pontas
    s = re.sub(r"\s+", " ", s)
    return s.strip(" ,;.-")

def _flex_regex_escape(s: str) -> str:
    """
    Cria uma regex flexível a partir de uma string.
    Ex: "TERMO DE ABERTURA" vira r"TERMO\s+DE\s+ABERTURA"
    Isso permite encontrar o texto mesmo com quebras de linha ou espaços extras.
    """
    return r"\s+".join(re.escape(part) for part in s.split())

def _slice_after_heading(text: str, heading: str, window: int = 1200) -> str:
    """
    Retorna um recorte do texto logo após o título/heading (ex: 'TERMO DE ABERTURA'),
    limitado por 'window' caracteres para focar no parágrafo certo.
    """
    # MODIFICADO: Usa a nova função flex_regex_escape
    heading_regex = _flex_regex_escape(heading)
    m = re.search(heading_regex, text, flags=re.IGNORECASE)
    # FIM DA MODIFICAÇÃO

    if not m:
        return ""
    start = m.end()
    return text[start:start+window]

def get_numero_contrato(text: str) -> str:
    """
    Extrai somente 'NNN/AAAA' do número do contrato.
    Casos:
      TERMO DE CONaTRATO EMERGENCIAL ... N° 371/2023-...
      TERMO DE CONTRATO EMERGENCIAL ... N° 488/2023 - DAL/...
      ... N° 851/2022
    Fallback: 'CONTRATO ... N° NNN/AAAA'
    """
    # alvo principal (TERMO DE CONTRATO EMERGENCIAL ...)
    padrao1 = r"TERMO\s+DE\s+CONTRATO\s+EMERGENCIAL\s+DE\s+OBRAS\s+E\s+SERVI[ÇC]OS\s+DE\s+ENGENHARIA\s*N[º°]?\s*([0-9]{1,4}/[0-9]{4})"
    m = re.search(padrao1, text, flags=re.IGNORECASE)
    if m:
        return m.group(1).strip()

    # fallback geral 'CONTRATO ... N° 123/2023'
    padrao2 = r"CONTRATO[^\n]{0,120}?N[º°]?\s*([0-9]{1,4}/[0-9]{4})"
    m2 = re.search(padrao2, text, flags=re.IGNORECASE)
    if m2:
        return m2.group(1).strip()

    return ""


from googleapiclient.discovery import build

# depois do creds/default() e do gspread:
drive = build('drive', 'v3', credentials=creds)

def _ensure_public_view_link(drive, file_id: str) -> str:
    """
    Garante que o arquivo tenha permissão 'anyone with the link' como leitor
    e retorna o webViewLink no formato .../file/d/<id>/view.
    """
    # 1) tenta ler o webViewLink atual
    meta = drive.files().get(fileId=file_id, fields="id, webViewLink, permissions").execute()
    link = meta.get("webViewLink")

    # 2) se já tem link, retorna
    if link:
        return link

    # 3) aplica permissão pública (se sua organização permitir)
    try:
        drive.permissions().create(
            fileId=file_id,
            body={"type": "anyone", "role": "reader"},
            fields="id",
        ).execute()
    except Exception:
        # pode falhar em domínios corporativos com políticas de compartilhamento
        pass

    # 4) tenta de novo pegar o webViewLink
    meta2 = drive.files().get(fileId=file_id, fields="id, webViewLink").execute()
    link2 = meta2.get("webViewLink")

    # 5) fallback: monta manualmente
    return link2 or f"https://drive.google.com/file/d/{file_id}/view"

def _get_folder_id_by_name(drive, folder_name: str) -> str:
    resp = drive.files().list(
        q=f"name = '{folder_name}' and mimeType = 'application/vnd.google-apps.folder' and 'root' in parents and trashed = false",
        fields="files(id,name)", pageSize=10
    ).execute()
    files = resp.get("files", [])
    return files[0]["id"] if files else ""


def _map_pdf_links_in_folder(drive, folder_id: str) -> dict:
    """
    Retorna dict {nome_arquivo.pdf: share_link} garantindo que tenha o /view.
    """
    name_to_link = {}
    page_token = None
    while True:
        resp = drive.files().list(
            q=f"'{folder_id}' in parents and mimeType='application/pdf' and trashed = false",
            fields="nextPageToken, files(id,name,webViewLink)",
            pageToken=page_token
        ).execute()

        for f in resp.get("files", []):
            file_id = f["id"]
            share_link = f.get("webViewLink") or f"https://drive.google.com/file/d/{file_id}/view"
            # Se quiser **forçar** que fique público para qualquer pessoa com o link:
            # share_link = _ensure_public_view_link(drive, file_id)
            name_to_link[f["name"]] = share_link

        page_token = resp.get("nextPageToken")
        if not page_token:
            break
    return name_to_link



def get_nome_empresa(text: str) -> str:
    """
    MODIFICADO:
    1. Normaliza o texto (com _norm_text) para remover caracteres "sujos"
    2. Usa regex mais flexível para parar em (,), (.), (;) ou 'inscrita'
    3. A busca pelo "TERMO DE ABERTURA" também é flexível (via _slice_after_heading)
    """

    # MODIFICADO: Normaliza o texto ANTES de qualquer busca
    texto_normalizado = _norm_text(text)
    # FIM DA MODIFICAÇÃO

    # 1) bloco após TERMO DE ABERTURA
    bloco = _slice_after_heading(texto_normalizado, "TERMO DE ABERTURA", window=2000)

    m = re.search(
        r"intenç(?:ão|ao)\s+de\s+instaurar\s+procedimento\s+notificat(?:ório|orio)\s+contra\s+(?:a\s+)?empresa\s+(.+?)(?=[,.;]|inscrita)",
        texto_normalizado,  # Usa o texto já normalizado
        flags=re.IGNORECASE | re.DOTALL
    )
    if m:
        return _clean_company_name(m.group(1))

    # 2) FALLBACK GENÉRICO COMBINADO (Novo)
    #    Este padrão agora junta suas duas ideias:
    #    - \s*[,;]?\s*: Permite que tenha "empresa NOME" OU "empresa, NOME"
    #    - (?=[,.;]|inscrita): Mantém a regra de parada original, que é mais segura
    #
    m = re.search(
        r"contra\s+(?:a\s+)?empresa\s*[,;]?\s*(.+?)(?=[,.;]|inscrita)",
        texto_normalizado,  # Usa o texto já normalizado
        flags=re.IGNORECASE | re.DOTALL
    )
    if m:
        return _clean_company_name(m.group(1))
    # 4) nada encontrado
    return "ERRO AO ENCONTRAR O NOME DA EMPRESA"


def get_cnpj_empresa(text: str) -> str:
    """
    Mantém a lógica antiga, mas:
    1) Tenta primeiro ancorado no nome da empresa (get_nome_empresa)
    2) Aceita . / - como separadores em qualquer ponto, com \s* (inclui quebras de linha)
    3) Ignora ruído entre 'sob n' e o número
    4) Se falhar, cai no padrão antigo (flex) para não perder recall
    """
    import re

    texto_normalizado = _norm_text(text)
    nome = get_nome_empresa(texto_normalizado)
    flags = re.IGNORECASE | re.DOTALL

    # --- util: nome -> regex flex (espaços, vírgulas/pontos opcionais, LTDAx) ---
    def _name_to_regex(n: str) -> str:
        # colapsa espaços, permite \s+ entre palavras e pontuação final opcional (LTDA, LTDAA, ME, EPP etc.)
        parts = [p for p in re.split(r"\s+", n.strip()) if p]
        # cada parte com pontuação final opcional
        parts = [re.escape(p) + r"[.,]?" for p in parts]
        # se terminar com LTDA (ou similar), aceita sufuxo de letras (LTDAA)
        if parts:
            parts[-1] = parts[-1].rstrip(r"[.,]?") + r"[A-Z]{0,2}[.,]?"
        return r"\s+".join(parts)

    # CNPJ super flex: aceita . / - (ou nada) entre blocos + \s* (quebras/espacos)
    CNPJ_FLEX = (
        r"("                       # captura o CNPJ como um todo
        r"\d{2}\s*[\.\-]?\s*"
        r"\d{3}\s*[\.\-]?\s*"
        r"\d{3}\s*[\./\-]?\s*"
        r"\d{4}\s*[\.\-\/]?\s*"
        r"\d{2}"
        r")"
    )

    MIN_FAZENDA = r"inscrita\s+no\s+minist[ée]rio\s+da\s+fazenda"
    # 'sob n' com qualquer ruído não-numérico depois (º, °, o, -, texto), antes dos dígitos
    SOB_VARIANTE = r"sob\s+(?:o\s+)?n[^\d]{0,10}\s*"

    # 1) Primeiro: padrão ancorado no nome
    if nome and "ERRO" not in nome.upper():
        nome_flex = _name_to_regex(nome)
        padrao_ancorado = re.compile(
            rf"empresa\s+{nome_flex}\s*,?\s*{MIN_FAZENDA}\s*{SOB_VARIANTE}{CNPJ_FLEX}",
            flags=flags
        )
        bloco = _slice_after_heading(texto_normalizado, "TERMO DE ABERTURA", window=2000) or ""
        m = padrao_ancorado.search(bloco) or padrao_ancorado.search(texto_normalizado)
        if m:
            cnpj_raw = re.sub(r"\s+", "", m.group(1))      # tira quebras/espaços
            digits = re.sub(r"\D", "", cnpj_raw)
            return (f"{digits[0:2]}.{digits[2:5]}.{digits[5:8]}/{digits[8:12]}-{digits[12:14]}"
                    if len(digits) == 14 else "ERRO AO ENCONTRAR O CNPJ")

    # 2) Fallback: teu padrão antigo (com \s*), mas com separadores flexíveis
    padrao_flex_antigo = re.compile(
        rf"{MIN_FAZENDA}\s*{SOB_VARIANTE}{CNPJ_FLEX}",
        flags=flags
    )
    bloco2 = _slice_after_heading(texto_normalizado, "TERMO DE ABERTURA", window=2000) or ""
    m2 = padrao_flex_antigo.search(bloco2) or padrao_flex_antigo.search(texto_normalizado)
    if not m2:
        return "ERRO AO ENCONTRAR O CNPJ"

    cnpj_raw = re.sub(r"\s+", "", m2.group(1))
    digits = re.sub(r"\D", "", cnpj_raw)
    return (f"{digits[0:2]}.{digits[2:5]}.{digits[5:8]}/{digits[8:12]}-{digits[12:14]}"
            if len(digits) == 14 else "ERRO AO ENCONTRAR O CNPJ")


def get_proa_notificatorio(text: str):
    padrao = r"\b(\d{2}\/\d{4}-\d{7}-\d)\b"
    ms = re.findall(padrao, text)
    return ms[0] if ms else ""


def get_proa_mae(text: str, proa_atual: str):
    padrao = r"\b(\d{2}\/\d{4}-\d{7}-\d)\b"
    all_proas = re.findall(padrao, text)
    candidates = [p for p in all_proas if p != proa_atual]

    def ano(p):
        try:
            return int(p.split("/")[0])
        except:
            return 99

    if not candidates:
        return ""
    candidates.sort(key=ano)
    return candidates[0]

def get_expediente_data(pdf_path: str, proa_notif: str) -> dict:
    """
    Lê SOMENTE a página 'EXPEDIENTE Nº {proa_notif}' e extrai:
      - tipo_penalidade  ('multa' | 'advertencia' | 'nao aplicacao de penalidade')
      - percentual_multa ('N%' entre 0 e 10)
      - divida_ativa     ('CFIL/RS', 'CADIN/RS' ou 'CFIL/RS; CADIN/RS')
      - penalidade_meses ('1 mês' / 'N meses')
      - quando_multa_aplicada (data dd/mm/aaaa do rodapé dessa página)
    Caso a página não exista, retorna ERR_MSG nos 4 campos e '' na data.
    """
    # 1) localizar página do expediente
    page_idx = _find_expediente_page_index(pdf_path, proa_notif)
    if page_idx < 0:
        return {
            "tipo_penalidade": ERR_MSG_TIPO_PENALIDADE,
            "percentual_multa": ERR_MSG_PERCENTUAL_MULTA,
            "impedimentos": ERR_MSG_IMPEDIMENTOS,
            "penalidade_meses": ERR_MSG_PENALIDADE_MESES,
            "data_penalizacao": ERR_MSG_DATA_PENALIZACAO,
        }

    # 2) texto da página + data do rodapé (seus helpers)
    page_text = _get_page_text(pdf_path, page_idx)
    quando = _footer_date_from_page(pdf_path, page_idx)

    # 3) parse dos 4 campos apenas do texto do EXPEDIENTE
    # tipo_penalidade
    if re.search(r"\bMULTA\b", page_text, re.IGNORECASE):
        tipo = "Multa"
    elif re.search(r"advert(ê|e)ncia", page_text, re.IGNORECASE):
        tipo = "Advertencia"
    elif re.search(r"n[aã]o\s+aplica(ç|c)[aã]o\s+de\s+penalidade", page_text, re.IGNORECASE):
        tipo = "Não Aplicação de penalidade"
    else:
        tipo = ERR_MSG_TIPO_PENALIDADE

    # percentual_multa (0–10) com '%'
    m = re.search(r"(?:aplicando\s+)?multa\s+(?:de\s+)?(\d{1,2})\s*%", page_text, re.IGNORECASE)
    if m and 0 <= int(m.group(1)) <= 10:
        per = f"{int(m.group(1))}%"
    else:
        per = ERR_MSG_PERCENTUAL_MULTA

    # divida_ativa (pode ter ambos)
    found = []
    if re.search(r"CFIL\/RS", page_text, re.IGNORECASE):  found.append("CFIL/RS")
    divida = "; ".join(found) if found else ERR_MSG_IMPEDIMENTOS

    # penalidade_meses (número ou por extenso)
    pen = ERR_MSG_PENALIDADE_MESES
    words = {"um":1,"uma":1,"dois":2,"duas":2,"tr[eê]s":3,"tres":3,"quatro":4,"cinco":5,"seis":6}
    m1 = re.search(r"prazo\s+de\s+\(?(\d{1,2})\)?\s+mes", page_text, re.IGNORECASE)
    if m1:
        v = int(m1.group(1)); pen = "1 mês" if v == 1 else f"{v} meses"
    else:
        m2 = re.search(r"prazo\s+de\s+\(([^)]+)\)\s+mes", page_text, re.IGNORECASE)
        if m2:
            w = re.sub(r"[^a-záéíóúâêôãõç]", "", m2.group(1).lower())
            for k,v in words.items():
                if re.search(k, w): pen = "1 mês" if v == 1 else f"{v} meses"; break
        if pen == ERR_MSG_PENALIDADE_MESES:
            m3 = re.search(r"prazo\s+de\s+([a-zçãõéê]+)\s+mes", page_text, re.IGNORECASE)
            if m3:
                w = m3.group(1).lower()
                for k,v in words.items():
                    if re.fullmatch(k, w): pen = "1 mês" if v == 1 else f"{v} meses"; break

    return {
        "tipo_penalidade": tipo,
        "percentual_multa": per,
        "divida_ativa": divida,
        "penalidade_meses": pen,
        "quando_multa_aplicada": quando,
    }


def get_tipo_penalidade(exp_text: str) -> str:
    if re.search(r"\bMULTA\b", exp_text, re.IGNORECASE):
        return "multa"
    if re.search(r"advert(ê|e)ncia", exp_text, re.IGNORECASE):
        return "advertencia"
    if re.search(r"n[aã]o\s+aplica(ç|c)[aã]o\s+de\s+penalidade", exp_text, re.IGNORECASE):
        return "nao aplicacao de penalidade"
    return ERR_MSG_TIPO_PENALIDADE


def get_percentual_multa(exp_text: str) -> str:
    # Dicionário para converter números por extenso em português
    words_to_num = {
        'zero': 0,
        'um': 1, 'uma': 1,
        'dois': 2, 'duas': 2,
        'três': 3,
        'quatro': 4,
        'cinco': 5,
        'seis': 6,
        'sete': 7,
        'oito': 8,
        'nove': 9,
        'dez': 10
    }

    # Regex para numérico: "multa de 05 %"
    m_num = re.search(r"(?:aplicando\s+)?multa\s+(?:de\s+)?(\d{1,2})\s*%", exp_text, re.IGNORECASE)
    # Regex para por extenso: "(cinco por cento)" logo após
    m_word = re.search(r"%\s*\(\s*([^)]+?)\s+por\s+cento\s*\)", exp_text, re.IGNORECASE)

    if not m_num:
        return ERR_MSG_PERCENTUAL_MULTA

    num_str = m_num.group(1).lstrip('0') or '0'  # Remove leading zero: "05" -> "5"
    num = int(num_str)

    if m_word:
        word = m_word.group(1).strip().lower()
        num_from_word = words_to_num.get(word)
        if num_from_word is not None:
            # Se bater (considerando leading zero), usa o valor
            if num == num_from_word:
                return f"{num}%"
            else:
                # Discrepância: prioriza por extenso como "mais sensato"
                return f"{num_from_word}%"

    # Se não houver por extenso ou não mapear, retorna o numérico ajustado
    return f"{num}%" if 0 <= num <= 10 else "ERRO NA PORCENTAGEM: MAIOR QUE 10%"

def get_impedimentos(exp_text: str) -> str:
    found = []
    if re.search(r"CFIL\/RS", exp_text, re.IGNORECASE):
        found.append("CFIL/RS")
    return "; ".join(found) if found else ERR_MSG_IMPEDIMENTOS


def get_penalidade_meses(exp_text: str) -> str:
    import re
    import unicodedata

    # Normaliza palavra para remover acentos e lower
    def normalize_word(w: str) -> str:
        w = unicodedata.normalize('NFKD', w.lower()).encode('ascii', 'ignore').decode('utf-8')
        return w.strip()

    # Dicionário expandido
    words = {
        "um": 1, "uma": 1,
        "dois": 2, "duas": 2,
        "tres": 3, "tre": 3,
        "quatro": 4,
        "cinco": 5,
        "seis": 6,
        "sete": 7,
        "oito": 8,
        "nove": 9,
        "dez": 10
    }

    # Padrão principal: captura frases antes, "prazo de" ou "por", número/extenso/combinado
    pat = r"(?:CFIL/RS\s*,\s*suspendendo\s+o\s+direito\s+de\s+licitar\s+ou\s+contratar\s+com\s+a\s+Administração\s*(?:,|pelo)?\s*)?(?:prazo\s+de|por)\s*(\d{1,2})?\s*\(\s*([^)]+)\s*\)?\s*meses?"
    m = re.search(pat, exp_text, re.IGNORECASE | re.DOTALL)
    if m:
        num_str = m.group(1)
        word_str = m.group(2)

        if num_str:
            num = int(num_str.lstrip('0') or '0')
            if word_str:
                w = normalize_word(word_str)
                for k, v in words.items():
                    if re.search(k, w):
                        # Verifica se bate; se não, usa extenso (mais sensato)
                        if num != v:
                            num = v
                        break
        elif word_str:
            w = normalize_word(word_str)
            for k, v in words.items():
                if re.search(k, w):
                    num = v
                    break
            else:
                return ERR_MSG_PENALIDADE_MESES
        else:
            return ERR_MSG_PENALIDADE_MESES

        return "1 mês" if num == 1 else f"{num} meses"

    # Fallbacks originais para padrões simples
    m1 = re.search(r"prazo\s+de\s+\(?(\d{1,2})\)?\s+mes", exp_text, re.IGNORECASE | re.DOTALL)
    if m1:
        v = int(m1.group(1).lstrip('0') or '0')
        return "1 mês" if v == 1 else f"{v} meses"

    m2 = re.search(r"prazo\s+de\s+\(([^)]+)\)\s+mes", exp_text, re.IGNORECASE | re.DOTALL)
    if m2:
        w = normalize_word(m2.group(1))
        for k, v in words.items():
            if re.search(k, w):
                return "1 mês" if v == 1 else f"{v} meses"

    m3 = re.search(r"prazo\s+de\s+([a-zçãõéê]+)\s+mes", exp_text, re.IGNORECASE | re.DOTALL)
    if m3:
        w = normalize_word(m3.group(1))
        for k, v in words.items():
            if re.fullmatch(k, w):
                return "1 mês" if v == 1 else f"{v} meses"

    return ERR_MSG_TIPO_PENALIDADE

def get_expediente_text_and_date(pdf_path: str, proa_notif: str) -> tuple[str, str]:
    idx = _find_expediente_page_index(pdf_path, proa_notif)
    if idx < 0:
        return "", ""
    exp_text = _get_page_text(pdf_path, idx)
    exp_text = _norm_text(exp_text)            # <-- aqui
    quando = _footer_date_from_page(pdf_path, idx)
    return exp_text, quando

def _norm_text(s: str) -> str:
    # normaliza espaços e hífens “estranhos”
    s = (s.replace("\xa0", " ")   # NBSP
           .replace("\u2009", " ") # thin space
           .replace("\u200a", " ")
           .replace("\u200b", "")  # zero-width
           .replace("–", "-")
           .replace("—", "-")
           .replace("-", "-"))     # non-breaking hyphen
    # colapsa espaços múltiplos
    s = re.sub(r"[ \t]+", " ", s)
    return s

def _build_proa_regex(proa_notif: str) -> str:
    # Ex: 19/1900-0050962-6 → permite espaços, quebras, hífens
    parts = re.split(r'[/-]', proa_notif)
    if len(parts) != 4:
        return re.escape(proa_notif)
    a, b, c, d = [p.strip() for p in parts]
    return rf"{a}\s*/\s*{b}\s*-\s*{c.lstrip('0')}\s*-\s*{d}"

def _find_expediente_page_index(pdf_path: str, proa_notif: str) -> int:
    import fitz, re

    # 1. Primeiro tenta pelo EXPEDIENTE N° (com seu _norm_text)
    proa_pat = _build_proa_regex(proa_notif)
    header_pat = rf"EXPEDIENTE.*?N[\sº°o\.\-\°]*{proa_pat}"
    pat_header = re.compile(header_pat, re.IGNORECASE | re.DOTALL)

    # 2. Fallback: frase padrão (flexível com \s+)
    frase_flex = r"Em\s+an[áa]lise\s+aos\s+autos\s+e\s+considerando\s+as\s+raz[õo]es\s+f[áa]ticas\s+e\s+contratuais"
    pat_frase = re.compile(frase_flex, re.IGNORECASE | re.DOTALL)

    try:
        doc = fitz.open(pdf_path)
        for i, page in enumerate(doc):
            txt = page.get_text("text")
            txt_norm = _norm_text(txt)

            # Estratégia 1: EXPEDIENTE N°
            if pat_header.search(txt_norm):
                return i

            # Estratégia 2: Frase (só se tiver o PROA na mesma página)
            if pat_frase.search(txt_norm) and proa_notif in txt_norm:
                return i

        doc.close()
    except Exception:
        pass
    return -1



def _get_page_text(pdf_path: str, page_index: int) -> str:
    import fitz
    doc = fitz.open(pdf_path)
    txt = doc[page_index].get_text("text")
    doc.close()
    return txt

def _footer_date_from_page(pdf_path: str, page_index: int, bottom_pct: float = 0.85) -> str:
    import fitz, re
    try:
        doc = fitz.open(pdf_path)
        page = doc[page_index]
        blocks = page.get_text("blocks")
        h = page.rect.height
        cutoff = h * bottom_pct
        cands = []
        for (x0,y0,x1,y1,txt,*_) in blocks:
            if y0 >= cutoff:
                for m in re.finditer(r"(\d{2}/\d{2}/\d{4})(?:\s+\d{2}:\d{2}:\d{2})?", txt):
                    cands.append((x0, y0, m.group(1)))
        doc.close()
        if not cands: return ""
        cands.sort(key=lambda t: (t[0], -t[1]))  # mais à esquerda
        return cands[0][2]
    except Exception:
        return ""


def get_quando_multa_aplicada(pdf_path: str, proa_notif: str) -> str:
    """
    Data mostrada no rodapé da página do EXPEDIENTE Nº {proa}.
    """
    idx = _find_expediente_page_index(pdf_path, proa_notif)
    if idx < 0:
        return ""
    return _footer_date_from_page(pdf_path, idx)

def get_ultima_atualizacao_processo(pdf_path_or_text):
    """
    Extrai a data da última atualização do processo.
    1) Tenta ler o rodapé da última página do PDF (campo à esquerda, ex: '09/07/2025 17:59:14').
    2) Se não encontrar, busca pela data mais recente no texto.
    """
    padrao_data = r"\b(\d{1,2}[\/\.]\d{1,2}[\/\.]\d{4})\b"

    # ======== tentativa 1: rodapé da última página ========
    try:
        if os.path.exists(pdf_path_or_text):  # é um arquivo PDF
            doc = fitz.open(pdf_path_or_text)
            page = doc[-1]
            blocks = page.get_text("blocks")
            page_h = page.rect.height
            cutoff_y = page_h * 0.85  # parte inferior (rodapé)
            datas_footer = []
            for b in blocks:
                x0, y0, x1, y1, txt = b[:5]
                if y0 >= cutoff_y:
                    for m in re.finditer(r"(\d{2}/\d{2}/\d{4})(?:\s+\d{2}:\d{2}:\d{2})?", txt):
                        datas_footer.append((x0, y0, m.group(1)))
            doc.close()
            if datas_footer:
                # escolhe a mais à esquerda (menor x0)
                datas_footer.sort(key=lambda t: (t[0], -t[1]))
                return datas_footer[0][2]
    except Exception:
        pass

    # ======== tentativa 2: busca no texto ========
    # se o argumento for texto, usa direto; senão tenta ler texto do PDF
    if os.path.exists(pdf_path_or_text):
        try:
            doc = fitz.open(pdf_path_or_text)
            text = "\n".join(page.get_text("text") for page in doc)
            doc.close()
        except Exception:
            text = ""
    else:
        text = pdf_path_or_text

    datas = re.findall(padrao_data, text)
    if not datas:
        return ""

    valid_dates = []
    for d_str in datas:
        d_str_norm = d_str.replace(".", "/")
        try:
            dd, mm, yyyy = map(int, d_str_norm.split("/"))
            valid_dates.append((datetime.date(yyyy, mm, dd), d_str_norm))
        except ValueError:
            continue
    if not valid_dates:
        return ""

    valid_dates.sort(key=lambda x: x[0], reverse=True)
    return valid_dates[0][1]



def get_data_analise_agora():
    hoje = datetime.datetime.now().strftime("%d/%m/%Y")
    return hoje

STATUS_SEM_CALCULO = {
    "advertencia",
    "nao aplicacao de penalidade",
    ERR_MSG_STATUS  # "ERRO: não encontrado o EXPEDIENTE"
}

def aplicar_regras_status(data: dict) -> dict:
    """
    Aplica regra de negócio:
    Se tipo_penalidade ∈ STATUS_SEM_CALCULO → limpa percentual, dívida, meses.
    """
    tipo = data.get("tipo_penalidade", "").lower().strip()

    if tipo in STATUS_SEM_CALCULO:
        data["percentual_multa"] = ""
        data["divida_ativa"] = ""
        data["penalidade_meses"] = ""

    return data

# ==========================
# EXTRAÇÃO DE CAMPOS (1 PDF)
# (Esta função permanece IDÊNTICA)
# ==========================
def extract_fields_from_pdf(pdf_path: str) -> dict:
    import re, time

    full_text = extract_pdf_text(pdf_path)
    tokens_estimados = estimate_tokens(full_text)
    proa_notif = get_proa_notificatorio(full_text)
    cnpj_empresa = get_cnpj_empresa(full_text)
    valor_contrato = ""
    valor_multa = ""

    # =========== STATUS DO PROCESSO (WEB) ===========
    status_proa = ""
    if proa_notif:
        # Remove tudo que não é dígito → 23/1900-0050521-5 → 23190000505215
        proa_num_raw = re.sub(r"\D", "", proa_notif)
        if proa_num_raw:
            print(f"🔎 Consultando status do PROA {proa_num_raw}...")
            time.sleep(3)  # delay para respeitar o servidor
            status_proa = get_situacao_processo_web(proa_num_raw) or ""
            print(f"→ Status retornado: {status_proa}\n")

    # ============= DEBUG OPCIONAL =============
    NOME_DO_ARQUIVO_QUE_FALHA = "ANALUZA"  # ajuste se quiser logar um PDF específico
    if NOME_DO_ARQUIVO_QUE_FALHA.lower() in pdf_path.lower():
        print(f"\n\n--- DEBUGGING PDF: {pdf_path} ---")
        texto_normalizado_debug = _norm_text(full_text)
        bloco_termo_abertura_debug = _slice_after_heading(texto_normalizado_debug, "TERMO DE ABERTURA", window=2000)
        print("--- BLOCO 'TERMO DE ABERTURA' (visto pelo script): ---")
        print(repr(bloco_termo_abertura_debug))
        print("-----------------------------------------------------\n\n")

    # ===== EXPEDIENTE =====
    exp_text, quando_aplicada = ("", "")
    if proa_notif:
        exp_text, quando_aplicada = get_expediente_text_and_date(pdf_path, proa_notif)

    if exp_text:
        tipo_penalidade  = get_tipo_penalidade(exp_text)
        percentual_multa = get_percentual_multa(exp_text)
        impedimentos     = get_impedimentos(exp_text)
        penalidade_meses = get_penalidade_meses(exp_text)
    else:
        tipo_penalidade  = ERR_MSG_TIPO_PENALIDADE
        percentual_multa = ERR_MSG_PERCENTUAL_MULTA
        impedimentos     = ERR_MSG_IMPEDIMENTOS
        penalidade_meses = ERR_MSG_PENALIDADE_MESES

    data = {
        "numero_contrato":             get_numero_contrato(full_text),
        "nome_empresa":                get_nome_empresa(full_text),
        "cnpj_empresa":                cnpj_empresa,
        "proa_notificatorio":          proa_notif,
        "proa_mae":                    get_proa_mae(full_text, proa_notif),
        "status_processo":             status_proa,                 # <-- status agora vem da web
        "valor_contrato_consolidado":  valor_contrato,  # -> vazio por enquanto
        "tipo_penalidade":             tipo_penalidade,
        "percentual_multa":            percentual_multa,
        "valor_multa":                 valor_multa, # -> vazio por enquanto
        "impedimentos":                impedimentos,
        "penalidade_meses":            penalidade_meses,
        "data_penalizacao":            quando_aplicada,
        "ultima_analise_feita":        get_data_analise_agora(),
        "ultima_atualizacao_processo": get_ultima_atualizacao_processo(pdf_path),
        #"token_input_consumo":         tokens_estimados, -> removido por enquanto
    }

    # Regras de negócio: limpa campos de multa quando não aplicável
    data = aplicar_regras_status(data)

    # Garante todas as colunas esperadas
    for col in COLUMNS:
        data.setdefault(col, "")

    return data




# ==========================
# FUNÇÃO: CARREGAR / CRIAR PLANILHA (MODIFICADA)
# ==========================
def load_or_create_gsheet(gc: gspread.Client, sheet_name: str, worksheet_name: str, columns: list[str]) -> tuple[
    pd.DataFrame, gspread.Worksheet]:
    """
    Carrega uma planilha Google Sheets. Se a aba (worksheet) não existir, cria.
    Retorna o DataFrame e o objeto worksheet.
    """
    try:
        # 1. Abre a Planilha (Spreadsheet)
        sh = gc.open(sheet_name)
    except gspread.exceptions.SpreadsheetNotFound:
        print(f"ERRO: Planilha '{sheet_name}' não encontrada.")
        print("Por favor, crie a planilha no Google Sheets e compartilhe com o e-mail da sua conta Colab.")
        raise

    try:
        # 2. Abre a Aba (Worksheet)
        ws = sh.worksheet(worksheet_name)
    except gspread.exceptions.WorksheetNotFound:
        # Cria a aba se não existir
        print(f"Aviso: Aba '{worksheet_name}' não encontrada. Criando...")
        ws = sh.add_worksheet(title=worksheet_name, rows=1, cols=len(columns))
        # Define o cabeçalho
        ws.update([columns])
        # Retorna um DF vazio, pois acabamos de criar
        return pd.DataFrame(columns=columns), ws

    # 3. Se a aba existe, carrega os dados para o pandas
    # Usamos gspread_dataframe para facilitar
    df = get_as_dataframe(ws, dtype=str)

    # 4. Garante que todas as colunas esperadas existam (mesma lógica de antes)
    for col in columns:
        if col not in df.columns:
            df[col] = pd.NA  # Use pd.NA para valores nulos consistentes

    # Converte pd.NA para string vazia "" para consistência com gspread
    df = df.fillna("")

    # 5. Reordena/filtra para manter apenas as colunas esperadas
    df = df[columns]

    return df, ws


# ==========================
# FUNÇÃO: ATUALIZAR/INSERIR LINHA
# (Esta função permanece IDÊNTICA)
# ==========================
def upsert_row(df: pd.DataFrame, row: dict) -> pd.DataFrame:
    """
    Se já existir mesma 'proa_notificatorio', atualiza.
    Senão, adiciona nova linha.
    Se não tem proa_notificatorio, não insere (evita linha vazia).
    """
    key = row.get("proa_notificatorio", "").strip()

    if key == "":
        print("⚠ Aviso: PDF ignorado porque não foi possível extrair proa_notificatorio.")
        return df

    # Converte a coluna do df para string para garantir a comparação
    df["proa_notificatorio"] = df["proa_notificatorio"].astype(str)

    if key in df["proa_notificatorio"].values:
        idx = df.index[df["proa_notificatorio"] == key][0]
        for col in COLUMNS:
            df.at[idx, col] = row.get(col, df.at[idx, col])
    else:
        df = pd.concat([df, pd.DataFrame([row])], ignore_index=True)

    return df


# ==========================
# PIPELINE COMPLETO (MODIFICADO)
# ==========================
def process_all_pdfs(gc: gspread.Client, pdf_dir=PDF_DIR, gsheet_name=GSHEET_NAME,
                     worksheet_name=GSHEET_WORKSHEET_NAME):
    """
    Função principal que orquestra todo o processo.
    Agora também aplica hyperlink no PROA se link do Drive existir.
    """
    # 1. Carrega / cria a planilha
    df, ws = load_or_create_gsheet(gc, gsheet_name, worksheet_name, COLUMNS)

    # 2. Mapear PDFs da pasta no Drive -> link
    folder_name = os.path.basename(PDF_DIR.rstrip("/"))
    folder_id = _get_folder_id_by_name(drive, folder_name)
    name_to_link = _map_pdf_links_in_folder(drive, folder_id) if folder_id else {}

    # 3. Processa PDFs
    for fname in os.listdir(pdf_dir):
        if not fname.lower().endswith(".pdf"):
            continue

        pdf_path = os.path.join(pdf_dir, fname)
        print(f"Processando: {pdf_path}")

        try:
            row = extract_fields_from_pdf(pdf_path)
            df = upsert_row(df, row)
        except Exception as e:
            print(f"ERRO ao processar o PDF {fname}: {e}")
            continue

    # 4. Mantém só linhas com PROA
    df = df[df["proa_notificatorio"].notna() & (df["proa_notificatorio"].str.strip() != "")].copy()

    # 5. Criar versão com hyperlinks
    df_to_write = df.copy()

    for i, row in df_to_write.iterrows():
        proa_val = row.get("proa_notificatorio", "").strip()
        fname = f"{proa_val}.pdf"  # você pode ajustar se o nome do arquivo for diferente
        texto = proa_val

        try:
            link = name_to_link.get(fname, "")
            if link and texto:
                df_to_write.at[i, "proa_notificatorio"] = f'=HYPERLINK("{link}"; "{texto}")'
        except Exception as e:
            # fallback: se algo der errado, deixa só o nome
            df_to_write.at[i, "proa_notificatorio"] = texto

    # 6. Escrever no Google Sheets
    print(f"Atualizando Google Sheet '{gsheet_name}'...")
    ws.clear()
    set_with_dataframe(ws, df_to_write, include_index=False, resize=True)

    # 7. Remover validação antiga
    sh = gc.open(gsheet_name)
    sheet_id = ws.id
    col_index = COLUMNS.index("percentual_multa") + 1
    col_0based = col_index - 1

    requests = {
        "requests": [
            {
                "setDataValidation": {
                    "range": {
                        "sheetId": sheet_id,
                        "startRowIndex": 0,
                        "startColumnIndex": col_0based,
                        "endColumnIndex": col_0based + 1
                    },
                    "rule": None
                }
            }
        ]
    }

    sh.batch_update(requests)
    print(f"Validação removida da coluna {chr(64 + col_index)} ✅")
    print("Planilha atualizada com sucesso! ✅")

    return df



In [ ]:
# ==========================
# EXECUÇÃO (atualizado p/ mostrar STATUS)
# ==========================
# 1) Garanta que os PDFs estão em PDF_DIR (ex.: /content/drive/MyDrive/processos_cibelle)
# 2) Garanta que você já rodou a CÉLULA DE AUTENTICAÇÃO e que a variável 'gc' existe.

from IPython.display import display

try:
    # Roda o pipeline completo (lê PDFs, extrai dados e atualiza a planilha)
    df_resultado = process_all_pdfs(gc)

    print("\n✅ Pipeline concluído. Amostra do DataFrame consolidado:")
    display(df_resultado.head(10))

    # ----------------------------
    # VISÃO FOCADA EM STATUS
    # ----------------------------
    cols_status = [
        "proa_notificatorio",
        "status_processo",
        "nome_empresa",
        "ultima_atualizacao_processo",
        "tipo_penalidade",
        "percentual_multa",
        "penalidade_meses",
        "divida_ativa",
    ]

    # Garante colunas caso estejam vazias
    for c in cols_status:
        if c not in df_resultado.columns:
            df_resultado[c] = ""

    # Tabela resumida: PROA x STATUS (ordenado por PROA)
    df_status = (
        df_resultado[cols_status]
        .copy()
        .sort_values(by=["status_processo", "proa_notificatorio"], na_position="last")
        .reset_index(drop=True)
    )

    print("\n📋 Status por processo (PROA):")
    display(df_status)

    # Contagem por status (distribuição)
    print("\n📊 Distribuição de status:")
    contagem = df_resultado["status_processo"].fillna("").replace("", "— (vazio)").value_counts()
    display(contagem.to_frame("quantidade"))

    # Lista rápida de processos sem status retornado
    sem_status = df_resultado[df_resultado["status_processo"].fillna("") == ""]
    if not sem_status.empty:
        print("\n⚠️ Processos sem status retornado (pode ser erro no site/consulta):")
        display(sem_status[["proa_notificatorio", "nome_empresa"]])

except NameError:
    print("\nERRO: A variável 'gc' não foi definida.")
    print("Por favor, rode a célula de autenticação do Google Colab (no início) antes de executar este bloco.")
except Exception as e:
    print(f"\nOcorreu um erro inesperado: {e}")
    print("Verifique se o nome da planilha está correto e se ela está compartilhada.")
